# 1 数据读取

#### 将DataFrame数据按行和列分别分割保存为csv数据文件来模拟各方持有的数据情况，读取csv数据分别为HDataFrame和VDataFrame类型变量

## 1.1 初始化

In [1]:
import secretflow as sf

# Check the version of your SecretFlow
print("The version of SecretFlow: {}".format(sf.__version__))

# In case you have a running secretflow runtime already.
sf.shutdown()

pyu_port = 16307
spu_port = 11666

cluster_config = {
    "parties": {
        "alice": {
            # replace with alice's real address.
            "address": "ecm-01:" + str(pyu_port),
            "listen_addr": "0.0.0.0:" + str(pyu_port),
        },
        "bob": {
            # replace with bob's real address.
            "address": "ecm-02:" + str(pyu_port),
            "listen_addr": "0.0.0.0:" + str(pyu_port),
        },
    },
    "self_party": "bob",
}

tls_config = {
    "ca_cert": "/home/beng003/certificate/alice_ca.crt",
    "cert": "/home/beng003/certificate/bob_server_cert.crt",
    "key": "/home/beng003/certificate/bob_server_key.key",
}


sf.init(address="ecm-02:6379", cluster_config=cluster_config, tls_config=tls_config)
alice, bob = sf.PYU("alice"), sf.PYU("bob")


from secretflow.data.horizontal import read_csv
from secretflow.security.aggregation import SecureAggregator
from secretflow.security.compare import SPUComparator
from secretflow.utils.simulation.datasets import load_dermatology

# aggr = SecureAggregator(alice, [alice, bob])


import spu

cluster_def = {
    "nodes": [
        {
            "party": "alice",
            # The address for other peers.
            "address": "ecm-01:" + str(spu_port),
            # The listen address of this node.
            # Optional. Address will be used if listen_address is empty.
            "listen_address": "0.0.0.0:" + str(spu_port),
        },
        {
            "party": "bob",
            "address": "ecm-02:" + str(spu_port),
            "listen_address": "0.0.0.0:" + str(spu_port),
        },
    ],
    "runtime_config": {
        "protocol": spu.spu_pb2.SEMI2K,
        "field": spu.spu_pb2.FM128,
        "sigmoid_mode": spu.spu_pb2.RuntimeConfig.SIGMOID_REAL,
    },
}

# link_desc = {
#     "connect_retry_times": 20,
#     "connect_retry_interval_ms": 5000,
#     "recv_timeout_ms": 25000,
#     "http_max_payload_size": 1048576,
#     "http_timeout_ms": 10000,
#     "throttle_window_size": 1024,
# }

link_desc = {
    "recv_timeout_ms": 6000,
}

print("*****************************************************Bob SPU")
spu_device = sf.SPU(cluster_def=cluster_def, link_desc=link_desc)

2024-08-13 18:04:59,819	INFO worker.py:1540 -- Connecting to existing Ray cluster at address: ecm-02:6379...
2024-08-13 18:04:59,835	INFO worker.py:1724 -- Connected to Ray cluster.
2024-08-13 18:04:59.861 INFO api.py:233 [bob] -- [Anonymous_job] Started rayfed with {'CLUSTER_ADDRESSES': {'alice': 'ecm-01:16307', 'bob': '0.0.0.0:16307'}, 'CURRENT_PARTY_NAME': 'bob', 'TLS_CONFIG': {'ca_cert': '/home/beng003/certificate/alice_ca.crt', 'cert': '/home/beng003/certificate/bob_server_cert.crt', 'key': '/home/beng003/certificate/bob_server_key.key'}}


The version of SecretFlow: 1.8.0b0


2024-08-13 18:05:00.840 INFO barriers.py:284 [bob] -- [Anonymous_job] Succeeded to create receiver proxy actor.
(ReceiverProxyActor pid=189371) 2024-08-13 18:05:00.836 INFO grpc_proxy.py:359 [bob] -- [Anonymous_job] ReceiverProxy binding port 16307, options: (('grpc.enable_retries', 1), ('grpc.so_reuseport', 0), ('grpc.max_send_message_length', 524288000), ('grpc.max_receive_message_length', 524288000), ('grpc.service_config', '{"methodConfig": [{"name": [{"service": "GrpcService"}], "retryPolicy": {"maxAttempts": 5, "initialBackoff": "5s", "maxBackoff": "30s", "backoffMultiplier": 2, "retryableStatusCodes": ["UNAVAILABLE"]}}]}'))...
(ReceiverProxyActor pid=189371) 2024-08-13 18:05:00.839 INFO grpc_proxy.py:379 [bob] -- [Anonymous_job] Successfully start Grpc service with credentials.
2024-08-13 18:05:01.804 INFO barriers.py:333 [bob] -- [Anonymous_job] SenderProxyActor has successfully created.
2024-08-13 18:05:01.805 INFO barriers.py:520 [bob] -- [Anonymous_job] Try ping ['alice'] at

*****************************************************Bob SPU


(raylet) [2024-08-13 18:05:14,582 E 132390 132414] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2024-08-13_17-11-20_281145_132313 is over 95% full, available space: 4231237632; capacity: 105089261568. Object creation will fail if spilling is required.
(raylet) [2024-08-13 18:05:24,593 E 132390 132414] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2024-08-13_17-11-20_281145_132313 is over 95% full, available space: 4234907648; capacity: 105089261568. Object creation will fail if spilling is required.
(raylet) [2024-08-13 18:05:34,600 E 132390 132414] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2024-08-13_17-11-20_281145_132313 is over 95% full, available space: 4231053312; capacity: 105089261568. Object creation will fail if spilling is required.
(raylet) [2024-08-13 18:05:44,607 E 132390 132414] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2024-08-13_17-11-20_281145_132313 is over 95% full, available space: 4233773056; capacity: 105089261568. Object 

(raylet) [2024-08-13 18:05:54,616 E 132390 132414] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2024-08-13_17-11-20_281145_132313 is over 95% full, available space: 4234764288; capacity: 105089261568. Object creation will fail if spilling is required.
(raylet) [2024-08-13 18:06:04,624 E 132390 132414] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2024-08-13_17-11-20_281145_132313 is over 95% full, available space: 4230946816; capacity: 105089261568. Object creation will fail if spilling is required.
(_run pid=135102) INFO:jax._src.xla_bridge:Unable to initialize backend 'cuda': 
(_run pid=135102) INFO:jax._src.xla_bridge:Unable to initialize backend 'rocm': module 'jaxlib.xla_extension' has no attribute 'GpuAllocatorConfig'
(_run pid=135102) INFO:jax._src.xla_bridge:Unable to initialize backend 'tpu': INTERNAL: Failed to open libtpu.so: libtpu.so: cannot open shared object file: No such file or directory
(raylet) [2024-08-13 18:06:14,631 E 132390 132414] (raylet) file_

## 1.2 查看Dataframe数据

In [2]:
import pandas as pd
from sklearn.datasets import load_iris

iris = load_iris(as_frame=True)
data = pd.concat([iris.data, iris.target], axis=1)
data["uid"] = data.index
data

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target,uid
0,5.1,3.5,1.4,0.2,0,0
1,4.9,3.0,1.4,0.2,0,1
2,4.7,3.2,1.3,0.2,0,2
3,4.6,3.1,1.5,0.2,0,3
4,5.0,3.6,1.4,0.2,0,4
...,...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,2,145
146,6.3,2.5,5.0,1.9,2,146
147,6.5,3.0,5.2,2.0,2,147
148,6.2,3.4,5.4,2.3,2,148


## 1.3 数据横向分割（每个参与方持有一定数量的完整数据）

In [3]:
# Horizontal partitioning.
h_alice, h_bob = data.iloc[:75, :], data.iloc[75:, :]

# Save to temporary files.
import os

temp_dir = "/home/beng003/python_project/sf-test/data/"

h_alice_path = os.path.join(temp_dir, 'h_alice.csv')
h_bob_path = os.path.join(temp_dir, 'h_bob.csv')
h_alice.to_csv(h_alice_path, index=False)
h_bob.to_csv(h_bob_path, index=False)

## 1.4 数据纵向分割（每个参与方持有一定数量的特征数据）

In [4]:
import numpy as np

# Vertical partitioning.
v_alice, v_bob = (
    data.iloc[:, np.r_[0:3, -1]],
    data.iloc[:, 3:],
)

# Save to temporary files.
v_alice_path = os.path.join(temp_dir, 'v_alice.csv')
v_bob_path = os.path.join(temp_dir, 'v_bob.csv')
v_alice.to_csv(v_alice_path, index=False)
v_bob.to_csv(v_bob_path, index=False)

## 1.5 读取横向分割的csv数据

In [5]:

from secretflow.data.horizontal import read_csv as h_read_csv
from secretflow.security.aggregation import SecureAggregator
from secretflow.security import SecureAggregator
from secretflow.security.compare import SPUComparator

# The aggregator and comparator are respectively used to aggregate
# or compare data in subsequent data analysis operations.
aggr = SecureAggregator(device=alice, participants=[alice, bob])

comp = SPUComparator(spu_device)
hdf = h_read_csv(
    {alice: h_alice_path, bob: h_bob_path},
    aggregator=aggr,
    comparator=comp,
)

hdf.drop(columns=["uid"], inplace=True)

2024-08-13 18:05:36.620 INFO proxy.py:187 [bob] -- [Anonymous_job] Create proxy actor <class 'secretflow.device.proxy.Actor_Masker'> with party alice.
2024-08-13 18:05:36.621 INFO proxy.py:187 [bob] -- [Anonymous_job] Create proxy actor <class 'secretflow.device.proxy.Actor_Masker'> with party bob.
2024-08-13 18:05:39.707 INFO proxy.py:187 [bob] -- [Anonymous_job] Create proxy actor <class 'secretflow.device.proxy.ActorPartitionAgent'> with party alice.
2024-08-13 18:05:39.708 INFO proxy.py:187 [bob] -- [Anonymous_job] Create proxy actor <class 'secretflow.device.proxy.ActorPartitionAgent'> with party bob.


## 1.6 读取纵向分割的csv数据

In [6]:
from secretflow.data.vertical import read_csv as v_read_csv

vdf = v_read_csv(
    {alice: v_alice_path, bob: v_bob_path},
    spu=spu_device,
    keys="uid",
    drop_keys="uid",
    psi_protocl="ECDH_PSI_2PC",
)

2024-08-13 18:05:51.187 INFO proxy.py:187 [bob] -- [Anonymous_job] Create proxy actor <class 'secretflow.device.proxy.ActorPartitionAgent'> with party alice.
2024-08-13 18:05:51.189 INFO proxy.py:187 [bob] -- [Anonymous_job] Create proxy actor <class 'secretflow.device.proxy.ActorPartitionAgent'> with party bob.


(SPURuntime(device_id=None, party=bob) pid=189755) [2024-08-13 18:05:51.172] [info] [launch.cc:164] LEGACY PSI config: {"psi_type":"ECDH_PSI_2PC","broadcast_result":true,"input_params":{"path":"/home/beng003/python_project/sf-test/data/v_bob.csv","select_fields":["uid"],"precheck":true},"output_params":{"path":"/home/beng003/python_project/sf-test/data/v_bob.csv.psi_output_45157","need_sort":true},"curve_type":"CURVE_25519","bucket_size":1048576}
(SPURuntime(device_id=None, party=bob) pid=189755) [2024-08-13 18:05:51.172] [info] [bucket_psi.cc:400] bucket size set to 1048576
(SPURuntime(device_id=None, party=bob) pid=189755) [2024-08-13 18:05:51.175] [info] [bucket_psi.cc:252] Begin sanity check for input file: /home/beng003/python_project/sf-test/data/v_bob.csv, precheck_switch:true
(SPURuntime(device_id=None, party=bob) pid=189755) [2024-08-13 18:05:51.177] [info] [csv_checker.cc:135] Executing duplicated scripts: LC_ALL=C sort --parallel=8 --buffer-size=1G --stable selected-keys.4b6

In [7]:
from typing import Union
import pandas as pd
from secretflow.data.horizontal import HDataFrame
from secretflow.data.vertical import VDataFrame

def table_statistics_vh(
    table: Union[pd.DataFrame, VDataFrame, HDataFrame]
) -> pd.DataFrame:
    """Get table statistics for a pd.DataFrame, VDataFrame or HDataFrame.

    Args:
        table: Union[pd.DataFrame, VDataFrame, HDataFrame]
    Returns:
    """
    assert isinstance(
        table, (pd.DataFrame, VDataFrame, HDataFrame)
    ), "table must be a pd.DataFrame, VDataFrame or HDataFrame"
    index = table.columns
    result = pd.DataFrame(index=index)
    result["datatype"] = table.dtypes
    result["total_count"] = table.shape[0]
    result["count(non-NA count)"] = table.count()
    result["count_na(NA count)"] = table.isna().sum()
    result["na_ratio"] = table.isna().sum() / table.shape[0]
    result["min"] = table.min(numeric_only=True)
    result["max"] = table.max(numeric_only=True)
    result["mean"] = table.mean(numeric_only=True)

    result["sum"] = table.sum(numeric_only=True)

    return result

In [8]:
# from secretflow.stats.table_statistics import table_statistics

pd.set_option("display.max_rows", None)
data_stats = table_statistics_vh(vdf)
data_stats

,datatype,total_count,count(non-NA count),count_na(NA count),na_ratio,min,max,mean,sum
sepal length (cm),float64,150,150,0,0.0,4.3,7.9,5.843333,876.5
sepal width (cm),float64,150,150,0,0.0,2.0,4.4,3.057333,458.6
petal length (cm),float64,150,150,0,0.0,1.0,6.9,3.758000,563.7
petal width (cm),float64,150,150,0,0.0,0.1,2.5,1.199333,179.9
target,int64,150,150,0,0.0,0.0,2.0,1.000000,150.0


In [9]:
table_statistics_vh(hdf)

,datatype,total_count,count(non-NA count),count_na(NA count),na_ratio,min,max,mean,sum
sepal length (cm),float64,150,150,0,0.0,4.3,7.9,5.843333,876.499996
sepal width (cm),float64,150,150,0,0.0,2.0,4.4,3.057333,458.599998
petal length (cm),float64,150,150,0,0.0,1.0,6.9,3.758000,563.699997
petal width (cm),float64,150,150,0,0.0,0.1,2.5,1.199333,179.899998
target,int64,150,150,0,0.0,0.0,2.0,1.000000,150.000000


In [10]:
sf.shutdown()

2024-08-13 18:06:18.564 INFO api.py:342 [bob] -- [Anonymous_job] Shutdowning rayfed intendedly...
2024-08-13 18:06:18.565 INFO api.py:356 [bob] -- [Anonymous_job] Wait for data sending.
2024-08-13 18:06:18.567 INFO message_queue.py:72 [bob] -- [Anonymous_job] Notify message polling thread[DataSendingQueueThread] to exit.
2024-08-13 18:06:18.614 INFO message_queue.py:102 [bob] -- [Anonymous_job] The message polling thread[DataSendingQueueThread] was exited.
2024-08-13 18:06:18.615 INFO message_queue.py:72 [bob] -- [Anonymous_job] Notify message polling thread[ErrorSendingQueueThread] to exit.
2024-08-13 18:06:18.658 INFO message_queue.py:102 [bob] -- [Anonymous_job] The message polling thread[ErrorSendingQueueThread] was exited.
2024-08-13 18:06:18.660 INFO barriers.py:469 [bob] -- [Anonymous_job] Stop sender proxy actor...
2024-08-13 18:06:18.662 INFO barriers.py:473 [bob] -- [Anonymous_job] Sender proxy actor stoped.
2024-08-13 18:06:18.663 INFO api.py:384 [bob] -- [Anonymous_job] Shu